In [3]:
import pyBigWig
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import gzip
import shutil
import os
import subprocess
import matplotlib.image as mpimg
import math 

In [ ]:
peaks_file = "/home/z5628505/Data/Cellcount/FLI1/Reference/FLI1_PEER300K_JV3_peaks.narrowPeak"
output_prefix = "FLI1300K" # i thin this is not used
output_directory = "/home/z5628505/Data/Cellcount/FLI1/Results"  # Define the directory here
bigwig_folder = "/home/z5628505/Data/Cellcount/FLI1/BW"

Creating an expanded 

In [5]:
# Ensure output directory is an absolute path
output_directory = os.path.abspath(output_directory)

# Convert peaks_file to absolute path (if it's not already)
peaks_file = os.path.abspath(peaks_file)

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Construct the output file path
output_file = os.path.join(output_directory, f"{os.path.splitext(os.path.basename(peaks_file))[0]}_expanded.bed")

# Print the output path for debugging
print(f"Saving expanded peaks to: {output_file}")

# Load BED file (assuming no header)
df = pd.read_csv(peaks_file, sep="\t", header=None)


# Compute summit position
summit = df[1] + df[4]

# Expand regions symmetrically
df[1] = (summit-500).clip(lower=0)
df[2] = summit +750


# Save the expanded BED file
df.to_csv(output_file, sep="\t", header=False, index=False)

print(f"Expanded peaks saved to {output_file}")


Saving expanded peaks to: /home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV3_peaks_expanded.bed
Expanded peaks saved to /home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV3_peaks_expanded.bed


Make katrix with bigwig files and defined region

In [6]:
bw_files = [os.path.join(bigwig_folder, f) for f in os.listdir(bigwig_folder) if f.endswith(".bw")]

print("Found BigWig files:", bw_files)



for bw_file in bw_files:
    bw_name = os.path.basename(bw_file).replace(".bw", "")
    matrix_file = os.path.join(output_directory, f"{bw_name}_Matrix.gz")

    # Run computeMatrix
    command =[
        "computeMatrix", "reference-point",
        "-R", output_file, "-S", bw_file,
        "--skipZeros", "--outFileName", matrix_file
]
    subprocess.run(command, check=True)


Found BigWig files: ['/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER100K_JV2.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER300K_JV6.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER500K_JV1.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER20M_JT132.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FL1_PEER75K_MN31.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER37K_MN32.bw', '/home/z5628505/Data/Cellcount/FLI1/BW/FLI1_PEER300K_JV3.bw']


Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_14297, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_14297, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_14297, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_14297, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_14297, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_10328, due to being absent in the computeMatrix output.
Skipping FLI1_PEER300K_JV3_peak_

In [7]:

# Define output directory where matrix files are stored
matrix_directory = output_directory  # This is where matrices are stored
heatmap_directory = output_directory  # This is where heatmaps will be saved

# Get all .gz files in the output directory
matrix_files = [os.path.join(matrix_directory, f) for f in os.listdir(matrix_directory) if f.endswith(".gz")]

print("Found matrix files:", matrix_files)

for matrix_file in matrix_files:
    print(f"Processing {matrix_file}")

    # Read matrix file directly from .gz
    with gzip.open(matrix_file, "rt", encoding="utf-8", errors="replace") as f:
        df = pd.read_csv(f, sep="\t", comment='@', header=None)

    # Check the first few rows to inspect the structure
    print(f"First few rows of {matrix_file}:")
    print(df.head())

    # Create the heatmap
    matrix_name = os.path.basename(matrix_file).replace("_matrix.gz", "")
       # Define both PNG and SVG output paths
    heatmap_output_png = os.path.join(heatmap_directory, f"{matrix_name}_heatmap.png")
    heatmap_output_svg = os.path.join(heatmap_directory, f"{matrix_name}_heatmap.svg")

    # Command for PNG output
   # command_png = [
       # "plotHeatmap",
       # "-m", matrix_file,  # Path to the matrix file
       # "-out", heatmap_output_png,  # Output path for PNG
        #"--colorMap", "RdYlBu",  # Color map
    #]

    # Command for SVG output
    command_svg = [
        "plotHeatmap",
        "-m", matrix_file,  # Path to the matrix file
        "-out", heatmap_output_svg,  # Output path for SVG
        "--colorMap", "RdYlBu",  # Color map
    ]

    # Run both commands
    for command in [command_svg]:
        print(f"Running command: {' '.join(command)}")  # Debugging info
        result = subprocess.run(command, check=False, capture_output=True, text=True)
        print(result.stderr)  # Print any errors

    

Found matrix files: ['/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV6_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV3_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FL1_PEER75K_MN31_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER37K_MN32_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER500K_JV1_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER100K_JV2_Matrix.gz', '/home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER20M_JT132_Matrix.gz']
Processing /home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV6_Matrix.gz
First few rows of /home/z5628505/Data/Cellcount/FLI1/Results/FLI1_PEER300K_JV6_Matrix.gz:
    0       1       2                         3      4   5          6    \
0  chr1   28907   30157  FLI1_PEER300K_JV3_peak_1  447.0   .   5.132830   
1  chr1  199245  200495  FLI1_PEER300K_JV3_peak_2  357.0   .  10.265700   
2  chr1  369277  370527  FLI1_PEER300K_JV3_peak_3   34.0   .  10


# Define output directory where heatmap files are stored
heatmap_directory = output_directory  # Same directory where heatmaps were saved

# Get all .png heatmap images
heatmap_files = [os.path.join(heatmap_directory, f) for f in os.listdir(heatmap_directory) if f.endswith("_heatmap.png")]

# Set the number of columns (e.g., 3 columns per row)
n_column = 6
n_row = math.ceil(len(heatmap_files) / n_column)  # Calculate the required number of rows

# Create a subplot grid to fit all the heatmaps using gridspec for finer control
fig = plt.figure(figsize=(10 * n_column, 6 * n_row))  # Adjust size
gs = fig.add_gridspec(n_row, n_column)  # Create the grid

# Loop through each heatmap file and plot it on a different subplot
for i, heatmap_file in enumerate(heatmap_files):
    ax = fig.add_subplot(gs[i // n_column, i % n_column])  # Select the correct grid cell
    img = mpimg.imread(heatmap_file)  # Read the image
    ax.imshow(img)  # Display the image on the corresponding axis
    ax.axis('off')  # Hide axes (optional)
    ax.set_title(f"Heatmap {i+1}")  # Set the title for each heatmap

# Adjust layout to reduce space between subplots
plt.tight_layout(pad=0.5)  # Reduces the space between subplots

# Save the combined heatmap image
combined_heatmap_output = os.path.join(heatmap_directory, "combined_heatmap.png")
plt.savefig(combined_heatmap_output, dpi=300, bbox_inches="tight")

# Show the combined heatmap
plt.show()

print(f"Saved combined heatmap: {combined_heatmap_output}")